In [24]:
pip install --upgrade scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/7b/0e/25d6b5678ed3c7e12bc94d047d0e9492e89cc78b7ea0034ac0f1cf2ff304/scikit_learn-1.4.1.post1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB 495.5 kB/s eta 0:00:22
   ---------------------------------------- 0.1/10.6 MB 939.4 kB/s eta 0:00:12
    --------------------------------------- 0.2/10.6 MB 1.1 MB/s eta 0:00:10
   - -------------------------------------- 0.3/10.6 MB 1.4 MB/s eta 0:00:08
   - -------------------------------------- 0.4/10.6 MB 1.6 MB/s eta 0:00:07
   -- ------------------------------------- 0.6/10.6 MB 2.0 MB/s eta 0:00:06
   -- ------------------------------------- 0.7/10.6 MB 2.0 MB/s eta 0:00:06
   --- ------------------------------------ 1.0/10.6 MB 2.4 MB/s eta 0:00:05
   --- ---------

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\imkha\\anaconda3\\Lib\\site-packages\\~klearn\\datasets\\_svmlight_format_fast.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
import pandas as pd

In [3]:
data=pd.read_csv("C:\\Users\\imkha\\Desktop\\30 days of ml\\Sentiment_analyzer\\train.csv",encoding='Windows-1252')

# EDA

In [4]:
import matplotlib.pyplot as plt
import seaborn as sb

In [5]:
time_mapping={'morning':1,'noon':2,'night':3}
data['Time_encoded']=data['Time of Tweet'].map(time_mapping)

In [6]:
age_mapping={'0-20':1,'21-30':2,'31-45':3,'46-60':4,'60-70':5,'70-100':6}
data['age_encoded']=data['Age of User'].map(age_mapping)
data.head(3)

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²),Time_encoded,age_encoded
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60,1,1
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105,2,2
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18,3,3


# data preprocessing

In [7]:
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import ClassifierMixin
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

## Imputing

In [8]:
data.isnull().sum()

textID              0
text                1
selected_text       1
sentiment           0
Time of Tweet       0
Age of User         0
Country             0
Population -2020    0
Land Area (Km²)     0
Density (P/Km²)     0
Time_encoded        0
age_encoded         0
dtype: int64

In [9]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(fill_value='',strategy='constant')
data=pd.DataFrame(imputer.fit_transform(data),columns=data.columns)
data.isnull().sum()

textID              0
text                0
selected_text       0
sentiment           0
Time of Tweet       0
Age of User         0
Country             0
Population -2020    0
Land Area (Km²)     0
Density (P/Km²)     0
Time_encoded        0
age_encoded         0
dtype: int64

## Stemming, Removing Stopwords

In [10]:
port_stem = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [11]:
data['content']=data['selected_text'].apply(stemming)
data.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²),Time_encoded,age_encoded,content
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60,1,1,respond go
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105,2,2,sooo sad
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18,3,3,bulli
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164,1,4,leav alon
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26,2,5,son


In [12]:
selected_features=['selected_text','Time_encoded','age_encoded']
target_feature='sentiment'

In [13]:
vectorizer=TfidfVectorizer()
X = vectorizer.fit_transform(data['content'])

X

<27481x13290 sparse matrix of type '<class 'numpy.float64'>'
	with 105974 stored elements in Compressed Sparse Row format>

In [14]:
Y=data[target_feature].values

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y)

In [16]:
clf=SVC()

In [17]:
from sklearn.metrics import accuracy_score,r2_score,confusion_matrix

In [18]:
clf.fit(X_train,Y_train)
Y_pred=clf.predict(X_test)

In [19]:
print(accuracy_score(Y_test,Y_pred))
print(confusion_matrix(Y_test,Y_pred))

0.8104420593050755
[[1061  433   62]
 [ 102 2051   71]
 [  52  322 1343]]


In [20]:
text_input = input('Enter text:')
text_input = stemming(text_input)

# Convert the single string into a list before fitting
vectorized_input = vectorizer.transform([text_input])

# Now you can use the vectorized input to make predictions
prediction = clf.predict(vectorized_input)
print(prediction)

Enter text:this phone is so laggy
['neutral']


In [21]:
import pickle

In [22]:
pickle.dump(clf,open('C:\\Users\\imkha\\Desktop\\30 days of ml\\Sentiment_analyzer\\model.sav','wb'))

In [23]:
pickle.dump(vectorizer,open('C:\\Users\\imkha\\Desktop\\30 days of ml\\Sentiment_analyzer\\vectorizer_model.sav','wb'))